In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/24 23:56:13 WARN Utils: Your hostname, DESKTOP-4QJNQQU resolves to a loopback address: 127.0.1.1; using 172.17.233.51 instead (on interface eth0)
22/08/24 23:56:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 23:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Inspecting the Yellow dataset

In [4]:
yellow = spark.read.parquet('../../ADS_Project_1/raw_data/tlc_yellow')
yellow.limit(10)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [5]:
yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [15]:
print(f'Shape of yellow dataset is {yellow.count()} rows x {len(yellow.columns)} columns')

Shape of yellow dataset is 12671164 rows x 19 columns


# Inspecting HVFHV dataset 

In [6]:
hvfhv = spark.read.parquet('../../ADS_Project_1/raw_data/hvfhv')
hvfhv.limit(10)

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0003,B03404,B03404,2022-01-01 00:05:31,2022-01-01 00:05:40,2022-01-01 00:07:24,2022-01-01 00:18:28,170,161,1.18,664,24.9,0.0,0.75,2.21,2.75,0.0,0.0,23.03,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:19:27,2022-01-01 00:22:08,2022-01-01 00:22:32,2022-01-01 00:30:12,237,161,0.82,460,11.97,0.0,0.36,1.06,2.75,0.0,0.0,12.32,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:43:53,2022-01-01 00:57:37,2022-01-01 00:57:37,2022-01-01 01:07:32,237,161,1.18,595,29.82,0.0,0.89,2.65,2.75,0.0,0.0,23.3,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:15:36,2022-01-01 00:17:08,2022-01-01 00:18:02,2022-01-01 00:23:05,262,229,1.65,303,7.91,0.0,0.24,0.7,2.75,0.0,0.0,6.3,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:25:45,2022-01-01 00:26:01,2022-01-01 00:28:01,2022-01-01 00:35:42,229,141,1.65,461,9.44,0.0,0.28,0.84,2.75,0.0,0.0,7.44,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:34:44,2022-01-01 00:36:52,2022-01-01 00:38:50,2022-01-01 00:51:32,263,79,4.51,762,17.67,0.0,0.53,1.57,2.75,0.0,0.0,12.25,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:47:51,2022-01-01 00:52:00,2022-01-01 00:53:25,2022-01-01 01:08:56,113,140,3.68,931,16.68,0.0,0.5,1.48,2.75,0.0,0.0,12.75,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:06:21,2022-01-01 00:06:58,2022-01-01 00:08:58,2022-01-01 00:23:01,151,75,2.77,843,14.41,0.0,0.43,1.28,0.0,0.0,4.0,11.47,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:27:54,2022-01-01 00:30:26,2022-01-01 00:32:25,2022-01-01 00:44:15,263,229,2.04,710,10.64,0.0,0.32,0.94,2.75,0.0,0.0,9.55,N,N,,N,N
HV0003,B03404,B03404,2022-01-01 00:44:59,2022-01-01 00:48:23,2022-01-01 00:50:23,2022-01-01 01:15:30,237,169,8.79,1507,107.56,0.0,0.83,2.45,2.75,0.0,0.0,23.67,N,N,,N,N


In [7]:
hvfhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [16]:
print(f'Shape of hvfhv dataset is {hvfhv.count()} rows x {len(hvfhv.columns)} columns')

Shape of hvfhv dataset is 66976983 rows x 24 columns


# Preprocessing Steps -


## 1. Typecasting
Making sure that all columns are appropriately typed

## 2. Getting rid of unrelated data (as per Data Dictionary)
We won't care about some fields for the purposes of our analysis, so need to drop them

## 3. Handling of Missing Values
If NaN / null values are present, need to deal with them appropriately - imputation, deletion, etc

## 4. Outlier Removal / Analysis
Outliers have to be dealt with appropriately, as they can skew data


### Typecasting

There are several columns that we need to adjust the type for. They'll be listed below with their justifications
- VendorID: Inspection of the data revealed that this only takes integer values, so it should be changed to an int
- passenger_count: Should be changed to int since # of people is an integer
- RatecodeID: Similar to VendorID

In [8]:
# I want to check if any taxi drivers have reported half a person -
columns = ['passenger_count']
for column in columns:
    sdf.select(column).distinct().show()
    
# Can confirm that only integers are reported

NameError: name 'sdf' is not defined

# How come this code doesn't work?

sdf.select(column).distinct().show()

In [ ]:
columns = ['VendorID', 'passenger_count', 'RatecodeID']
for column in columns:
    sdf = sdf.withColumn(
        column,
        F.col(column).astype('INT')
    )

In [ ]:
sdf.printSchema()

### Converting PU/DOLocation to Geometry 
Code attributed to tute2

In [ ]:
sdf.select(F.min('fare_amount'))

In [ ]:
aggregated_results = sdf \
                    .groupBy("passenger_count") \
                    .agg(
                        F.mean("total_amount").alias("avg_trip_amount_usd"),
                        F.max("trip_distance").alias("max_trip_distance_miles")
                    ) \
                    .orderBy("passenger_count")

aggregated_results.show()